In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import random
from collections import defaultdict

import torch.nn.functional as F



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_indexes(arr, value):
    indexes = []
    for i in range(len(arr)):
        if arr[i] == value:
            indexes.append(i)
    return indexes

def get_length_per_class(dataloader, classes):
    class_counts = defaultdict(int)
    total = 0
    for batch in dataloader:
        _, labels = batch 
        labels = labels.numpy().tolist()
        for label in labels:
            class_counts[label] += 1
            total +=1

    class_counts = dict(sorted(class_counts.items()))
    for class_label, count in class_counts.items():
        print(f"Class {classes[class_label]}: {count} samples out of {total}")
def load_data(data_dir,
                           batch_size,
                           data_type,
                           noise_type,
                           noise_percentage,                           
                           transform,                           
                           data_percentage=1,
                           show_classes = False, random_seed=21):
    
    if noise_type == "None":
        noise_type = ""
        noise_percentage = ""
    else:
        noise_type = "/" + str(noise_type)
        noise_percentage = "/" + str(noise_percentage)
    path = data_dir + noise_type + "/" + data_type + noise_percentage
    print("path: ", path)
    dataset = ImageFolder(root=path, transform=transform)
    original_classes = dataset.classes 
    num_samples = len(dataset)
    indices = list(range(num_samples))

    labels = dataset.targets
    class_to_idx = dataset.class_to_idx
    needed_length = int(num_samples*data_percentage/100)
    expected_length_per_class = int(needed_length/len(original_classes))
    print(f"needed_length: {needed_length}, expected_length_per_class: {expected_length_per_class}")
    if data_percentage != 100:
        new_indices = []
        for key, value in class_to_idx.items():
            all_indixes_of_class = get_indexes(labels, value)
            new_indices.extend(all_indixes_of_class[:expected_length_per_class])
    else:
        new_indices = indices
    length_dataset = len(new_indices)
    print("length of final dataset:", length_dataset)

    
    # sampler = SubsetRandomSampler(new_indices)

    dataloader = DataLoader(dataset, sampler=new_indices, batch_size=batch_size)

    if show_classes:
        get_length_per_class(dataloader, original_classes)
        
    random.shuffle(new_indices)

   
    dataloader = DataLoader(dataset, sampler=new_indices, batch_size=batch_size)

    return dataloader, length_dataset, original_classes

l_storage = []
l_storage_per_epoch = []
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        
        l_storage_per_epoch = []
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0
            l = 0

            
            for inputs, labels in dataloaders[phase]:
                inputs =  torch.tensor(inputs).to(device)
                labels = torch.tensor(labels).to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    if not isinstance(outputs, torch.Tensor):
                        outputs = outputs.logits
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                l += len(inputs)
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                print(f"{epoch} epoch, correct: {running_corrects.item()}/{l} = {round(running_corrects.item()/l, 2)}")
                key = str(running_corrects) + '/' + str(l)
                l_storage_per_epoch.append({key:round(running_corrects.item()/l, 2)})

            
#             if phase == 'train':
#                 scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            l_storage.append(l_storage_per_epoch)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    model.load_state_dict(best_model_wts)
    return model
    
    







# Training alexnet

In [3]:
transform = transforms.Compose([
    transforms.Resize((227, 227)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4680, 0.4647, 0.3441], std=[0.2322, 0.2272, 0.2394]) 
])   

noise_type = "gaussian_noise"
noise_percentage = 10
data_percentage = 100
total_size = 21000

train_size = data_percentage*total_size/100
data_dir = '/kaggle/input/vegetableimages/vegetable_images'

train_loader, train_size, classes = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, valid_size, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader_with_noise, _, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = noise_type,
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataloaders = {'train':  train_loader, 
               'val': valid_loader
               }
dataloaders_with_noise = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }


test_loader,test_size_, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "gaussian_noise",
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


test_loader_without_noise, _, _ = load_data(data_dir =data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataset_sizes = {'train':  train_size, 
        'val': valid_size,
        'test': test_size_
       }




path:  /kaggle/input/vegetableimages/vegetable_images/train
needed_length: 15000, expected_length_per_class: 1000
length of final dataset: 15000
path:  /kaggle/input/vegetableimages/vegetable_images/validation
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/validation/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/test/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/test
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000


In [14]:
dataset_sizes

{'train': 15000, 'val': 3000, 'test': 3000}

In [15]:
alexnet = models.alexnet(pretrained=True)

In [16]:
in_features = alexnet._modules['classifier'][-1].in_features
out_features = len(classes)
alexnet._modules['classifier'][-1] = nn.Linear(in_features, out_features, bias=True)
print(alexnet._modules['classifier'])

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=15, bias=True)
)


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
alexnet = alexnet.to(device)

cuda


In [14]:
F = nn.functional
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

       
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            
            for inputs, labels in dataloaders[phase]:
                inputs =  torch.tensor(inputs).to(device)
                labels = torch.tensor(labels).to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs,_ = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

    
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    model.load_state_dict(best_model_wts)
    return model

In [19]:
for param in alexnet.parameters():
    param.requires_grad = True

In [20]:
criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(alexnet.parameters(), lr=0.001)


step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

alexnet = train_model(alexnet, criterion, optimizer, step_lr_scheduler, num_epochs=15)

Epoch 0/14
----------


/tmp/ipykernel_28/2272575281.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs =  torch.tensor(inputs).to(device)
/tmp/ipykernel_28/2272575281.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


train Loss: 0.4875 Acc: 0.8967
val Loss: 1.9262 Acc: 0.3857

Epoch 1/14
----------
train Loss: 0.1976 Acc: 0.9492
val Loss: 0.9692 Acc: 0.6803

Epoch 2/14
----------
train Loss: 0.1099 Acc: 0.9717
val Loss: 0.6022 Acc: 0.7907

Epoch 3/14
----------
train Loss: 0.0778 Acc: 0.9794
val Loss: 0.4343 Acc: 0.8557

Epoch 4/14
----------
train Loss: 0.0602 Acc: 0.9824
val Loss: 0.3366 Acc: 0.8803

Epoch 5/14
----------
train Loss: 0.0499 Acc: 0.9871
val Loss: 0.2628 Acc: 0.9100

Epoch 6/14
----------
train Loss: 0.0427 Acc: 0.9881
val Loss: 0.2289 Acc: 0.9230

Epoch 7/14
----------
train Loss: 0.1049 Acc: 0.9667
val Loss: 0.0658 Acc: 0.9793

Epoch 8/14
----------
train Loss: 0.0696 Acc: 0.9794
val Loss: 0.0576 Acc: 0.9800

Epoch 9/14
----------
train Loss: 0.0645 Acc: 0.9822
val Loss: 0.0509 Acc: 0.9827

Epoch 10/14
----------
train Loss: 0.0580 Acc: 0.9829
val Loss: 0.0467 Acc: 0.9840

Epoch 11/14
----------
train Loss: 0.0545 Acc: 0.9851
val Loss: 0.0423 Acc: 0.9863

Epoch 12/14
----------
t

In [24]:
torch.save(alexnet, "alexnet_without_noise.pth")

# Training VGG16

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4680, 0.4647, 0.3441], std=[0.2322, 0.2272, 0.2394]) 
])   

noise_type = "gaussian_noise"
noise_percentage = 10
data_percentage = 100
total_size = 21000

train_size = data_percentage*total_size/100
data_dir = '/kaggle/input/vegetableimages/vegetable_images'

train_loader, train_size, classes = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, valid_size, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader_with_noise, _, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = noise_type,
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataloaders = {'train':  train_loader, 
               'val': valid_loader
               }
dataloaders_with_noise = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }


test_loader,test_size_, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "gaussian_noise",
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


test_loader_without_noise, _, _ = load_data(data_dir =data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataset_sizes = {'train':  train_size, 
        'val': valid_size,
        'test': test_size_
       }




path:  /kaggle/input/vegetableimages/vegetable_images/train
needed_length: 15000, expected_length_per_class: 1000
length of final dataset: 15000
path:  /kaggle/input/vegetableimages/vegetable_images/validation
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/validation/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/test/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/test
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000


In [8]:
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.parameters():
    param.requires_grad = True

In [12]:
in_features = vgg16._modules['classifier'][-1].in_features
out_features = len(classes)
vgg16._modules['classifier'][-1] = nn.Linear(in_features, out_features, bias=True)
print(vgg16._modules['classifier'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
vgg16 = vgg16.to(device)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=15, bias=True)
)
cuda


In [14]:
criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(vgg16.parameters(), lr=0.001)


step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

vgg16 = train_model(vgg16, criterion, optimizer, step_lr_scheduler, num_epochs=15)

Epoch 0/14
----------


/tmp/ipykernel_28/1032719542.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs =  torch.tensor(inputs).to(device)
/tmp/ipykernel_28/1032719542.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


train Loss: 0.5577 Acc: 0.8909
val Loss: 2.1363 Acc: 0.3773

Epoch 1/14
----------
train Loss: 0.1993 Acc: 0.9503
val Loss: 1.0769 Acc: 0.6483

Epoch 2/14
----------
train Loss: 0.0929 Acc: 0.9738
val Loss: 0.5622 Acc: 0.8043

Epoch 3/14
----------
train Loss: 0.0669 Acc: 0.9801
val Loss: 0.3054 Acc: 0.8837

Epoch 4/14
----------
train Loss: 0.0459 Acc: 0.9869
val Loss: 0.1762 Acc: 0.9337

Epoch 5/14
----------
train Loss: 0.0357 Acc: 0.9893
val Loss: 0.1139 Acc: 0.9573

Epoch 6/14
----------
train Loss: 0.0315 Acc: 0.9904
val Loss: 0.1485 Acc: 0.9437

Epoch 7/14
----------
train Loss: 0.0727 Acc: 0.9761
val Loss: 0.0459 Acc: 0.9830

Epoch 8/14
----------
train Loss: 0.0537 Acc: 0.9835
val Loss: 0.0329 Acc: 0.9887

Epoch 9/14
----------
train Loss: 0.0487 Acc: 0.9851
val Loss: 0.0303 Acc: 0.9907

Epoch 10/14
----------
train Loss: 0.0436 Acc: 0.9866
val Loss: 0.0253 Acc: 0.9920

Epoch 11/14
----------
train Loss: 0.0403 Acc: 0.9874
val Loss: 0.0222 Acc: 0.9933

Epoch 12/14
----------
t

In [15]:
torch.save(vgg16, "vgg16_without_noise.pth")

# RESNET

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4680, 0.4647, 0.3441], std=[0.2322, 0.2272, 0.2394]) 
])   

noise_type = "gaussian_noise"
noise_percentage = 10
data_percentage = 100
total_size = 21000

train_size = data_percentage*total_size/100
data_dir = '/kaggle/input/vegetableimages/vegetable_images'

train_loader, train_size, classes = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, valid_size, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader_with_noise, _, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = noise_type,
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataloaders = {'train':  train_loader, 
               'val': valid_loader
               }
dataloaders_with_noise = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }


test_loader,test_size, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "gaussian_noise",
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


test_loader_without_noise, _, _ = load_data(data_dir =data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataset_sizes = {'train':  train_size, 
        'val': valid_size,
        'test': test_size
       }


dataset_sizes

path:  /kaggle/input/vegetableimages/vegetable_images/train
needed_length: 15000, expected_length_per_class: 1000
length of final dataset: 15000
path:  /kaggle/input/vegetableimages/vegetable_images/validation
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/validation/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/gaussian_noise/test/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages/vegetable_images/test
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000


{'train': 15000, 'val': 3000, 'test': 3000}

In [7]:
resnet = models.resnet101(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = True

num_features = resnet.fc.in_features
num_classes = 15  
resnet.fc = nn.Linear(num_features, num_classes)
resnet.to(device)

print(resnet.fc)

Linear(in_features=2048, out_features=15, bias=True)


In [8]:
criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(resnet.parameters(), lr=0.001)


step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

resnet = train_model(resnet, criterion, optimizer, step_lr_scheduler, num_epochs=15)

Epoch 0/14
----------


/tmp/ipykernel_28/3239572470.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs =  torch.tensor(inputs).to(device)
/tmp/ipykernel_28/3239572470.py:126: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


0 epoch, correct: 2/64 = 0.03
0 epoch, correct: 4/128 = 0.03
0 epoch, correct: 10/192 = 0.05
0 epoch, correct: 10/256 = 0.04
0 epoch, correct: 13/320 = 0.04
0 epoch, correct: 16/384 = 0.04
0 epoch, correct: 21/448 = 0.05
0 epoch, correct: 27/512 = 0.05
0 epoch, correct: 31/576 = 0.05
0 epoch, correct: 39/640 = 0.06
0 epoch, correct: 44/704 = 0.06
0 epoch, correct: 51/768 = 0.07
0 epoch, correct: 62/832 = 0.07
0 epoch, correct: 70/896 = 0.08
0 epoch, correct: 83/960 = 0.09
0 epoch, correct: 93/1024 = 0.09
0 epoch, correct: 102/1088 = 0.09
0 epoch, correct: 110/1152 = 0.1
0 epoch, correct: 117/1216 = 0.1
0 epoch, correct: 131/1280 = 0.1
0 epoch, correct: 138/1344 = 0.1
0 epoch, correct: 154/1408 = 0.11
0 epoch, correct: 168/1472 = 0.11
0 epoch, correct: 183/1536 = 0.12
0 epoch, correct: 197/1600 = 0.12
0 epoch, correct: 209/1664 = 0.13
0 epoch, correct: 233/1728 = 0.13
0 epoch, correct: 252/1792 = 0.14
0 epoch, correct: 272/1856 = 0.15
0 epoch, correct: 289/1920 = 0.15
0 epoch, correct: 

In [5]:
for inputs, labels in dataloaders['train']:
    print(labels)



tensor([ 8,  3,  6, 10, 14,  1,  4, 13,  3,  4, 11,  5,  5,  2,  9, 14,  5,  4,
         8,  1, 11, 14,  0, 10,  1,  6,  3,  8,  5,  3,  7,  1,  6,  8,  7,  8,
         0,  2, 14,  6,  1,  6,  2,  3,  3, 12,  7,  4, 13, 10,  1, 11, 10,  8,
         8,  6,  1,  3,  7,  4, 14, 11, 14,  1])
tensor([ 6,  7,  9,  2, 14, 12, 13,  2,  2,  0,  6,  7,  5,  5,  7,  3,  4, 13,
         8,  2,  0, 14, 14,  5,  7,  9,  0,  9, 10,  7,  3,  2, 11,  1, 11,  9,
         2,  8,  5, 13, 14,  9,  3,  6,  2,  7, 13,  1,  4, 14, 14, 13,  1,  5,
         5,  2, 11,  9, 10,  6, 14,  7,  2,  1])
tensor([ 9,  2, 11, 13, 12,  0,  6, 11, 12,  1,  6,  1,  9,  8,  9, 14,  7,  0,
         2, 11,  0, 10,  8,  7,  4,  2,  5, 13,  1,  1,  3, 13,  7,  3, 11,  7,
        14,  8,  8, 14,  1,  2, 14, 10, 13, 12,  6,  2, 14, 12,  0, 13,  3,  6,
         9,  3, 11,  0,  9,  0,  6, 10,  6,  1])
tensor([ 5, 13, 14, 10,  8, 10, 12, 13,  7,  1, 12,  9,  8,  7,  0,  8,  7,  8,
         8,  1,  2,  9, 12,  7,  8,  6,  4,  1,  9,  

KeyboardInterrupt: 

In [9]:
torch.save(resnet, "resnet_without_noise.pth")